<a href="https://colab.research.google.com/github/DelightInLinght/Detection_System/blob/main/Overfitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d priyanka369/deepfake-cropped-frames

Dataset URL: https://www.kaggle.com/datasets/priyanka369/deepfake-cropped-frames
License(s): unknown
 96% 473M/493M [00:04<00:00, 153MB/s]
100% 493M/493M [00:04<00:00, 118MB/s]


In [ ]:
!unzip deepfake-cropped-frames.zip

In [5]:
import tensorflow as tf
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation (optional) to improve generalization
train_datagen = ImageDataGenerator(
                         rescale=1./255,
                         shear_range=0.2,
                         zoom_range=0.2,
                         horizontal_flip=True,
                         rotation_range=20,
                         width_shift_range=0.2,
                         height_shift_range=0.2
                         )
validation_datagen = ImageDataGenerator(rescale=1./255)

# Prepare training and validation data generators
train_generator = train_datagen.flow_from_directory(
    r"/content/content/Dataset/Train",
    target_size=(150,150),  # Adjust for your model's input size
    batch_size=64,  # Experiment with batch size
    class_mode='binary'
)
validation_generator = validation_datagen.flow_from_directory(
    r"/content/content/Dataset/Valid",
    target_size=(150,150),
    batch_size=64,
    class_mode='binary'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    r"/content/content/Dataset/Valid",
    target_size=(150,150),
    batch_size=64,
    class_mode='binary'
)



Found 173760 images belonging to 2 classes.
Found 28736 images belonging to 2 classes.
Found 28736 images belonging to 2 classes.


In [19]:
from keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Add, Dropout
from keras.layers import Input, Dense, BatchNormalization
from keras.models import Model
from keras.applications import MobileNetV2

# Input Layer
input_layer = Input(shape=(150,150, 3))

# Inception Module
base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=input_layer, classes=1)

# Global Average Pooling
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Fully Connected Layers
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(16, activation='relu')(x)
x = Dropout(0.5)(x)

# Output Layer
output_layer = Dense(1, activation='sigmoid')(x)

# Model
model = Model(inputs=input_layer, outputs=output_layer)

# Freeze the layers of model

base_model.trainable = False
for layer in model.layers:
    print(layer.name, layer.trainable)


input_7 False
Conv1 False
bn_Conv1 False
Conv1_relu False
expanded_conv_depthwise False
expanded_conv_depthwise_BN False
expanded_conv_depthwise_relu False
expanded_conv_project False
expanded_conv_project_BN False
block_1_expand False
block_1_expand_BN False
block_1_expand_relu False
block_1_pad False
block_1_depthwise False
block_1_depthwise_BN False
block_1_depthwise_relu False
block_1_project False
block_1_project_BN False
block_2_expand False
block_2_expand_BN False
block_2_expand_relu False
block_2_depthwise False
block_2_depthwise_BN False
block_2_depthwise_relu False
block_2_project False
block_2_project_BN False
block_2_add False
block_3_expand False
block_3_expand_BN False
block_3_expand_relu False
block_3_pad False
block_3_depthwise False
block_3_depthwise_BN False
block_3_depthwise_relu False
block_3_project False
block_3_project_BN False
block_4_expand False
block_4_expand_BN False
block_4_expand_relu False
block_4_depthwise False
block_4_depthwise_BN False
block_4_depthwi

In [20]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 75, 75, 32)           864       ['input_7[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 75, 75, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 75, 75, 32)           0         ['bn_Conv1[0][0]']      

In [10]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# L2 Regularization (Weight Decay)
regularizer = regularizers.l2(0.001)  # Adjust the regularization strength as needed

# Create and compile the model
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    batch_size=64,
    steps_per_epoch=len(train_generator),
    validation_steps=len(validation_generator),
    epochs=10,
    callbacks=[early_stopping]
)

model.save("MobileNetV2.h5")


Epoch 1/10
2715/2715 [==============================] - 1073s 395ms/step - loss: 0.6969 - accuracy: 0.6121 - val_loss: 0.7442 - val_accuracy: 0.5390
Epoch 2/10
2715/2715 [==============================] - 1072s 395ms/step - loss: 0.6609 - accuracy: 0.6462 - val_loss: 0.7397 - val_accuracy: 0.5333
Epoch 3/10
1760/2715 [==================>...........] - ETA: 6:05 - loss: 0.6508 - accuracy: 0.6585

In [ ]:
#TinyConvNeXt (124,124,3); Epoch 1/10
# 2715/2715 [==============================] - 1253s 446ms/step - loss: 0.6890 - accuracy: 0.5551 - val_loss: 0.6907 - val_accuracy: 0.5390
#Epoch 2/10
# 2715/2715 [==============================] - 1253s 446ms/step - loss: 0.6790 - accuracy: 0.5651 - val_loss: 0.6807 - val_accuracy: 0.5090



# MobileNetV2 (224,224,3); Epoch 1/10
#2715/2715 [==============================] - 875s 310ms/step - loss: 0.2976 - accuracy: 0.8650 - val_loss: 2.8650 - val_accuracy: 0.5823
#Epoch 1/10
#2715/2715 [==============================] - 875s 310ms/step - loss: 0.2976 - accuracy: 0.8650 - val_loss: 2.8650 - val_accuracy: 0.5823
#Epoch 2/10
#2715/2715 [==============================] - 807s 297ms/step - loss: 0.1927 - accuracy: 0.9155 - val_loss: 1.2040 - val_accuracy: 0.7773
#Epoch 3/10
#2715/2715 [==============================] - 800s 295ms/step - loss: 0.1624 - accuracy: 0.9301 - val_loss: 1.3427 - val_accuracy: 0.7721

# InceptionV3 (299,299);Epoch 1/10
#2715/2715 [==============================] - 4576s 2s/step - loss: 0.3263 - accuracy: 0.8473 - val_loss: 0.8555 - val_accuracy: 0.7612
#Epoch 2/10
#21/2715 [..............................] - ETA: 1:09:37 - loss: 0.2232 - accuracy: 0.9025

# Xception  (150,150);Epoch 1/10
#2715/2715 [==============================] - 1491s 535ms/step - loss: 0.2411 - accuracy: 0.8925 - val_loss: 0.6205 - val_accuracy: 0.8443
#Epoch 2/10
#71/2715 [..............................] - ETA: 22:39 - loss: 0.1579 - accuracy: 0.9357


# fine tuning = Xception
# Epoch 1/10
# 2715/2715 ━━━━━━━━━━━━━━━━━━━━ 4080s 2s/step - accuracy: 0.7830 - loss: 0.4420 - val_accuracy: 0.7191 - val_loss: 0.9746
# Epoch 2/10
# 2715/2715 ━━━━━━━━━━━━━━━━━━━━ 3955s 1s/step - accuracy: 0.8729 - loss: 0.2817 - val_accuracy: 0.7361 - val_loss: 1.1567


In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    metrics=['accuracy']
)